In [1]:
# from datasets import load
from torch.utils.data import Dataset, DataLoader
import json
import torch
file_dict = '../datasets/cmrc2018/'

articles = []

class CMRC(Dataset):
    def __init__(self, split) -> None:
        super().__init__()
        self.data = []
        if split == 'train':
            f = 'cmrc2018_train.json'
        elif split == 'dev':
            f = 'cmrc2018_dev.json'
        elif split == 'test':
            f = 'cmrc2018_trial.json'

        with open(file_dict + f, 'r') as file:
            d = json.loads(file.readlines()[0])
        for each in d:
            articles.append(each['context_text'])
            for qa in each['qas']:
                self.data.append([qa['query_text'], qa['answers'][0], len(articles)-1])

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

# if __name__ == '__main__':
#     cmrc = CMRC(split='train')
#     cmrc = CMRC(split='dev')
#     cmrc = CMRC(split='test')
#     print()


d:\install\Anaconda\envs\py3713\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tokenization import FullTokenizer
tokenizer = FullTokenizer('./my_vocab.txt')

In [3]:
def get_batch(sample):
    
    q = [tokenizer.tokenize(each[0]) for each in sample]
    c = [tokenizer.tokenize(articles[each[2]]) for each in sample]
    answers = [tokenizer.tokenize(str(each[1])) for each in sample]
    labels = []
    for i in range(len(sample)):
        for j in range(len(c[i])):
            if c[i][j:j+len(answers[i])] == answers[i]:
                labels.append([j, j+len(answers[i])])
                break
    if len(labels) < len(c):
        print('error!!')
    result = []
    for i in range(len(sample)):
        if len(c[i]) > 400:
            for j in range(0, len(c[i])-50, 350):
                if labels[i][0]<j and labels[i][1]<j:
                    result.append([c[i][j: j+400], q[i], [0, 0]])
                elif labels[i][0]<j and labels[i][1]>=j:
                    result.append([c[i][j: j+400], q[i], [0, labels[i][1]-(j)]])
                elif labels[i][0]>=j and labels[i][1]<j+400:
                    result.append([c[i][j: j+400], q[i], [labels[i][0]-(j), labels[i][1]-(j)]])
                elif labels[i][0]<j+400 and labels[i][1]>=j+400:
                    result.append([c[i][j: j+400], q[i], [labels[i][0]-(j), 399]])
                elif labels[i][0]>=j+400 and labels[i][1]>=j+400:
                    result.append([c[i][j: j+400], q[i], [0, 0]])
                if j+400 >= len(c[i]):
                    break
        else: 
            result.append([c[i], q[i], labels[i]])
    return result

In [4]:
from tqdm import tqdm
import numpy
def process(split):
    processed_set = []
    train_set = CMRC(split=split)
    dl = DataLoader(train_set, batch_size=8, collate_fn=get_batch, shuffle=True)
    for i, batch in enumerate(tqdm(dl, leave=True)):
        processed_set.extend(batch)
    numpy.save(f'./data4finetuning/{split}', processed_set)
process('train')
process('dev')
process('test')

100%|██████████| 1268/1268 [00:24<00:00, 52.02it/s]
d:\install\Anaconda\envs\py3713\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
100%|██████████| 126/126 [00:02<00:00, 49.22it/s]
